In [1]:

import string
import re
from nltk.tokenize import word_tokenize
import requests

In [2]:

file = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')
# corpus = file.read()



In [3]:
corpus = file
corpus.text[:1500]

'This is the 100th Etext file presented by Project Gutenberg, and\nis presented in cooperation with World Library, Inc., from their\nLibrary of the Future and Shakespeare CDROMS.  Project Gutenberg\noften releases Etexts that are NOT placed in the Public Domain!!\n\nShakespeare\n\n*This Etext has certain copyright implications you should read!*\n\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\nPROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE\nWITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE\nDISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS\nPERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED\nCOMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY\nSERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>\n\n*Project Gutenberg is proud to cooperate with The World Library*\nin the presentation of The Complete Works of William Shakespeare\nfor your

In [4]:
data = corpus.text.split('\n')
data[0]

'This is the 100th Etext file presented by Project Gutenberg, and'

In [5]:
len(data)
data = " ".join(data)
data[:1000]

'This is the 100th Etext file presented by Project Gutenberg, and is presented in cooperation with World Library, Inc., from their Library of the Future and Shakespeare CDROMS.  Project Gutenberg often releases Etexts that are NOT placed in the Public Domain!!  Shakespeare  *This Etext has certain copyright implications you should read!*  <<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM SHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS PROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE WITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE DISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS PERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED COMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY SERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>  *Project Gutenberg is proud to cooperate with The World Library* in the presentation of The Complete Works of William Shakespeare for your reading for educati

In [6]:
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

tokens = clean_text(data)
print(tokens[:50])

['this', 'is', 'the', 'etext', 'file', 'presented', 'by', 'project', 'gutenberg', 'and', 'is', 'presented', 'in', 'cooperation', 'with', 'world', 'library', 'inc', 'from', 'their', 'library', 'of', 'the', 'future', 'and', 'shakespeare', 'cdroms', 'project', 'gutenberg', 'often', 'releases', 'etexts', 'that', 'are', 'not', 'placed', 'in', 'the', 'public', 'domain', 'shakespeare', 'this', 'etext', 'has', 'certain', 'copyright', 'implications', 'you', 'should', 'read']


In [7]:
len (( tokens))

899788

In [8]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
    break

print(len(lines))

199951


In [9]:
lines[0]

'this is the etext file presented by project gutenberg and is presented in cooperation with world library inc from their library of the future and shakespeare cdroms project gutenberg often releases etexts that are not placed in the public domain shakespeare this etext has certain copyright implications you should read this'

In [10]:
tokens[50]

'this'

In [11]:
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [13]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]
X[10]

array([   11,  6743,    10,  6744,    15,   122,   486,   520,    47,
          65,   486,     5,     1,  2775,     2,   418, 13163,   398,
         455,   678, 13162,  2187,     9,    33,    12,  4811,    10,
           1,   848,  3823,   418,    22,   397,   236,   611,   485,
       13161,     6,    86,   610,    22,   299,   539,     5,     1,
         487,   442,     5,   364,   418])

In [14]:
vocab_size = len(tokenizer.word_index) + 1


In [15]:
y = to_categorical(y, num_classes=vocab_size)


In [16]:
seq_length = X.shape[1]
seq_length

50

In [17]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [18]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            658250    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 13165)             1329665   
Total params: 2,138,815
Trainable params: 2,138,815
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [21]:
model.fit(X, y, batch_size = 64, epochs = 2)


Epoch 1/2
3125/3125 [==============================] - 509s 163ms/step - loss: 6.4353 - accuracy: 0.0547
Epoch 2/2
3125/3125 [==============================] - 512s 164ms/step - loss: 6.1593 - accuracy: 0.0736


In [22]:
seed_text=lines[12343]
seed_text

'cannot dispraise but in a kind of praise naming thy name blesses an ill report o what a mansion have those vices got which for their habitation chose out thee where beautys veil doth cover every blot and all things turns to fair that eyes can see take heed dear heart'

In [23]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [25]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 1)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'and'